# Sandbox
For playing around during development.

In [1]:
%pylab inline
import pysd
print pysd.__version__
print pysd.__file__
import pandas as pd

Populating the interactive namespace from numpy and matplotlib
0.3.2
pysd/__init__.pyc


In [2]:
#mdl_file = 'tests/old_tests/vensim/Teacup.mdl'
#mdl_file = 'tests/test-models/tests/subscript_3d_arrays/test_subscript_3d_arrays.mdl'
# mdl_file = 'tests/test-models/tests/subscript_2d_arrays/test_subscript_2d_arrays.mdl'
#mdl_file = 'tests/test-models/tests/subscript_individually_defined_1_of_2d_arrays/subscript_individually_defined_1_of_2d_arrays.mdl'
#mdl_file = 'tests/test-models/tests/subscript_individually_defined_1d_arrays/subscript_individually_defined_1d_arrays.mdl'
mdl_file = '../mounir/pdatvensim8k_RW_v2_4.mdl'
model = pysd.read_vensim(mdl_file)

In [4]:
%%time
model.run(flatten_subscripts=True)

CPU times: user 18.6 s, sys: 85.6 ms, total: 18.7 s
Wall time: 18.7 s


,cumulative_admin_support_cost,cumulative_labor_and_personnel_costs,cumulative_net_indirect_crew_costs,cumulative_platform_manning_costs,cumulative_scenario_extra_hours__onboard__preventive,cumulative_scenario_extra_hours__onboard__corrective,cumulative_scenario_extra_hours__intermediate__preventive,cumulative_scenario_extra_hours__intermediate__corrective,cumulative_scenario_extra_hours__depot__preventive,cumulative_scenario_extra_hours__depot__corrective,...,total_maintenance_costs,total_os_costs,total_rdte_costs,total_sustaining_support_costs,total_unit_level_personnel_costs,total_unit_operating_costs,unconstrainted_maintenance_costs__onboard,unconstrainted_maintenance_costs__intermediate,unconstrainted_maintenance_costs__depot,wpn
0.000000,0.000000,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0,0.000000e+00,0,0,0,0
0.007812,0.000000,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0,0.000000e+00,0,0,0,0
0.015625,0.000000,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0,0.000000e+00,0,0,0,0
0.023438,0.000000,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0,0.000000e+00,0,0,0,0
0.031250,0.000000,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0,0.000000e+00,0,0,0,0
0.039062,0.000000,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0,0.000000e+00,0,0,0,0
0.046875,0.000000,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0,0.000000e+00,0,0,0,0
0.054688,0.000000,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0,0.000000e+00,0,0,0,0
0.062500,0.000000,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0,0.000000e+00,0,0,0,0
0.070312,0.000000,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0,0.000000e+00,0,0,0,0


In [4]:
model.run(return_timestamps=np.arange(0,1,1))

,cumulative_admin_support_cost,cumulative_labor_and_personnel_costs,cumulative_net_indirect_crew_costs,cumulative_platform_manning_costs,cumulative_scenario_extra_hours,data_total_os_costs,executed_work,maintenance_costs_by_level,maintenence_work_backlog,mpn,...,total_indirect_costs,total_investment_costs,total_maintenance_costs,total_os_costs,total_rdte_costs,total_sustaining_support_costs,total_unit_level_personnel_costs,total_unit_operating_costs,unconstrainted_maintenance_costs,wpn
0,0,0,0,0,"[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]]",0,"[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]]","[0.0, 0.0, 0.0]","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]]",0,...,0,0,0,0,0,0,0,0,"[0.0, 0.0, 0.0]",0


In [ ]:
for key, func in model.components._dfuncs.iteritems():
    print func
    print func()
